In [1]:
import pandas as pd
import numpy as np

from Code.UtilityFunctions.get_data_path import get_path
from Code.UtilityFunctions.run_query import run_query

___
___
___
# **DO NOT RUN ALL LINES BELOW. SOME YELP ENTITIES ARE VERY LARGE, AND YOU MIGHT RUN OUT OF RAM.**

In [47]:
business = pd.read_json(get_path("yelp_academic_dataset_business.json"), lines=True)
business['categories'] = business['categories'].str.split(', ', expand=False)

In [3]:
users = pd.read_json(get_path("yelp_academic_dataset_user.json"), lines=True)

In [15]:
reviews = pd.read_json(get_path("yelp_academic_dataset_review.json"), lines=True)
reviews = pd.merge(left=reviews, right=business, how='left', left_on='business_id', right_on='business_id')
#reviews = pd.merge(left=reviews, right=users, how='left', left_on='user_id', right_on='user_id')

In [ ]:
checkins = pd.read_json(get_path("yelp_academic_dataset_checkin.json"), lines=True)
checkins['date'] = checkins['date'].str.split(', ', expand=False)
checkins = checkins.explode('date')

In [3]:
tips = pd.read_json(get_path("yelp_academic_dataset_tip.json"), lines=True)

___
___
___
### How many different types of businesses are defined in Yelp?

In [2]:
sparql_query ="""

SELECT COUNT(DISTINCT(?category))
WHERE {
    ?business schema:category ?category .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,callret-0.value
0,1311


In [5]:
unique_categories = {category for sublist in business['categories'] if sublist for category in sublist}
len(unique_categories)

1311

### How many businesses of type "Restaurants" exist?

In [3]:
sparql_query = """

SELECT COUNT(DISTINCT(?business)) AS ?numberRestaurants
WHERE {
    ?business schema:category yelpcat:Restaurants .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,numberRestaurants.value
0,52268


In [7]:
counter = 0
for row in business['categories']:
    if row is not None and "Restaurants" in row:
        counter += 1
print(counter)

52268


### How many businesses of type Restaurants have been reviewed?

In [54]:
sparql_query = """

SELECT COUNT(DISTINCT(?business))
WHERE {
    ?review schema:about ?business .
    ?business schema:category yelpcat:Restaurants .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,callret-0.value
0,0


In [6]:
sparql_query = """

SELECT ?business
WHERE {
    ?business schema:category yelpcat:Restaurants .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,business.value
0,https://purl.archive.org/purl/yelp/yelp_entiti...
1,https://purl.archive.org/purl/yelp/yelp_entiti...
2,https://purl.archive.org/purl/yelp/yelp_entiti...
3,https://purl.archive.org/purl/yelp/yelp_entiti...
4,https://purl.archive.org/purl/yelp/yelp_entiti...
...,...
52263,https://purl.archive.org/purl/yelp/yelp_entiti...
52264,https://purl.archive.org/purl/yelp/yelp_entiti...
52265,https://purl.archive.org/purl/yelp/yelp_entiti...
52266,https://purl.archive.org/purl/yelp/yelp_entiti...


In [13]:
review_unique_business = reviews.drop_duplicates(subset=['business_id'])
mask = review_unique_business['categories'].apply(lambda x: "Restaurants" in x)
len(review_unique_business[mask])

NameError: name 'reviews' is not defined

### How many businesses have been reviewed?

In [7]:
sparql_query = """

SELECT COUNT(DISTINCT(?business))
WHERE {
    ?review schema:about ?business .
    ?review rdfs:Class schema:UserReview .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,callret-0.value
0,150346


In [32]:
review_unique_business = reviews.drop_duplicates(subset=['business_id'])
len(review_unique_business)

150346

### How many businesses have, on average, a rating of 4.5?

In [23]:
sparql_query = """
SELECT COUNT(DISTINCT(?business))
WHERE {
    ?business schema:aggregateRating ?rating .
    FILTER (?rating = 4.5) .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,callret-0.value
0,27181


In [49]:
business[business['stars'] == 4.5].groupby('stars')['stars'].count()

stars
4.5    27181
Name: stars, dtype: int64

### What is the average rating across businesses?

In [11]:
sparql_query = """
SELECT AVG(?rating) as ?averagerating
WHERE {
    ?business schema:aggregateRating ?rating .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,averagerating.type,averagerating.datatype,averagerating.value
0,typed-literal,http://www.w3.org/2001/XMLSchema#decimal,3.59672355766033


In [25]:
business['stars'].mean()

3.5967235576603303

### How many businesses have been reviewed in Santa Barbara, CA?

TO DO: Select only Santa Barbara: VALUES ?city {'Santa Barbara'} does not work

In [2]:
sparql_query = """
SELECT ?state ?city COUNT(?business) AS ?count_business
WHERE {
    ?business yelpont:locatedInState ?state .
    ?business yelpont:locatedInCity ?city .
    ?city rdfs:label ?city_label .
    VALUES ?city_label{'Santa Barbara}
    
}
GROUP BY ?state ?city
ORDER BY DESC(?count_business)
LIMIT 10

"""

run_query(query=sparql_query, as_dataframe=True)

ConnectionResetError: [Errno 104] Connection reset by peer

In [31]:
business_santa_barbara = business[business['city'] == "Santa Barbara"]
len(business_santa_barbara)

3829

What are the five most busy days, and for what business?

In [51]:
sparql_query = """
SELECT ?business ?year ?month ?day COUNT(?visit) as ?numberOfVisits
WHERE {
    ?business schema:checkinTime ?visit .
    BIND (day(?visit)  as ?day)
    BIND (month(?visit) as ?month)
    BIND (year(?visit) as ?year)
}
GROUP BY ?business ?year ?month ?day
ORDER BY DESC(COUNT(?visit))
LIMIT 5

"""

run_query(query=sparql_query, as_dataframe=True)

,business.value,year.value,month.value,day.value,numberOfVisits.value
0,https://purl.archive.org/purl/yelp/yelp_entiti...,2016,6,25,457
1,https://purl.archive.org/purl/yelp/yelp_entiti...,2015,8,30,285
2,https://purl.archive.org/purl/yelp/yelp_entiti...,2016,8,28,268
3,https://purl.archive.org/purl/yelp/yelp_entiti...,2016,8,28,263
4,https://purl.archive.org/purl/yelp/yelp_entiti...,2017,7,29,251


In [17]:
checkins["Day"] = checkins["date"].apply(lambda x: x.split("-")[2][:2])
checkins["Month"] = checkins["date"].apply(lambda x: x.split("-")[1])
checkins["Year"] = checkins["date"].apply(lambda x: x.split("-")[0])

checkins.value_counts(subset=["business_id", "Day", "Month", "Year"], sort=True, ascending=False).head(5)

business_id             Day  Month  Year
CySqUcNz8oPiQTu4EXTnig  25   06     2016    465
g50ImmCX3WY3koEDIzoKxg  30   08     2015    287
qfWWx0dVo1UuAhRfh03Dyw  28   08     2016    270
g50ImmCX3WY3koEDIzoKxg  28   08     2016    264
FBBeJO50xZiNIo3oFhAFRA  29   07     2017    254
dtype: int64

Which are the top 10 most visisted businesses?

In [52]:
sparql_query = """
SELECT ?business COUNT(?visit) AS ?count_visits
WHERE {
    ?business schema:checkinTime ?visit .
}
GROUP BY ?business 
ORDER BY DESC(COUNT(?visit))
LIMIT 10

"""

run_query(query=sparql_query, as_dataframe=True)

,business.value,count_visits.value
0,https://purl.archive.org/purl/yelp/yelp_entiti...,52129
1,https://purl.archive.org/purl/yelp/yelp_entiti...,40092
2,https://purl.archive.org/purl/yelp/yelp_entiti...,37553
3,https://purl.archive.org/purl/yelp/yelp_entiti...,37511
4,https://purl.archive.org/purl/yelp/yelp_entiti...,31163
5,https://purl.archive.org/purl/yelp/yelp_entiti...,29599
6,https://purl.archive.org/purl/yelp/yelp_entiti...,28917
7,https://purl.archive.org/purl/yelp/yelp_entiti...,21527
8,https://purl.archive.org/purl/yelp/yelp_entiti...,21478
9,https://purl.archive.org/purl/yelp/yelp_entiti...,18604


In [20]:
checkins.value_counts(subset=["business_id"], sort=True, ascending=False).head(10)

business_id           
-QI8Qi8XWH3D8y8ethnajA    52144
FEXhWNCMkv22qG04E83Qjg    40109
Eb1XmmLWyt_way5NNZ7-Pw    37562
c_4c5rJECZSfNgFj7frwHQ    37518
4i4kmYm9wgSNyF1b6gKphg    31168
8O35ji_yOMVJmZ6bl96yhQ    29606
VQcCL9PiNL_wkGf-uF3fjg    28927
ac1AeYqs8Z4_e2X5M3if2A    21542
QTbahs-GVuWYL5yfdjH34A    21487
ytynqOUb3hjKeJfRj5Tshw    18615
dtype: int64

How many people have written a review or tip on Yelp?
TO DO: Not correct result

In [63]:
sparql_query = """
SELECT COUNT(DISTINCT(?user)) AS ?countUsers
WHERE {
    ?user rdfs:Class schema:Person .
    ?review schema:author ?user .
}
"""

run_query(query=sparql_query, as_dataframe=True)

,count_users.value
0,1987897


In [7]:
reviews.drop_duplicates(subset=['user_id']).shape[0]

1987929

How many users have 10 friends?
TO DO: Only select the Counterusers?

In [104]:
sparql_query = """
SELECT COUNT(DISTINCT(?user)) AS ?countUsers
WHERE {
    ?user rdfs:Class schema:Person .
    ?user schema:knows ?friend .
}
GROUP BY ?user
HAVING (COUNT(?friend) = 10)
"""

run_query(query=sparql_query, as_dataframe=True)

,countUsers.value
0,1
1,1
2,1
3,1
4,1
...,...
13574,1
13575,1
13576,1
13577,1


In [11]:
users["amountFriends"] = users["friends"].apply(lambda x: len(x.split(",")))
users[users["amountFriends"] == 10].shape[0]

13579

How many friends does a user have on average?
TO DO: Is not giving correct result

In [73]:
sparql_query = """
SELECT AVG(?numberOfFriends) as ?averageFriends
WHERE { SELECT COUNT(?friend) as ?numberOfFriends
WHERE {
    ?user rdfs:Class schema:Person .
    ?user schema:knows ?friend .
}
GROUP BY ?user
}
"""

run_query(query=sparql_query, as_dataframe=True)

,averageFriends.value
0,94.853611046508483


In [12]:
# How many friends does a user have on average?
users["amountFriends"].mean()

53.375011381374385

How many users have authored 10 reviews?

In [84]:
sparql_query = """
SELECT COUNT(DISTINCT(?user)) AS ?countUsers
WHERE {
SELECT ?user COUNT(?review) as ?numberOfReviews
WHERE {
    ?user rdfs:Class schema:Person .
    ?review schema:author ?user .
}
GROUP BY ?user
HAVING (COUNT(?review) = 10)
}
"""

run_query(query=sparql_query, as_dataframe=True)

,countUsers.value
0,14119


In [17]:
# Count how many users have authored 10 reviews
reviews.groupby("user_id").size().reset_index(name="count").query("count == 10").shape[0]

14119

How many reviews did users make in May 2018?

In [98]:
sparql_query = """
SELECT ?year ?month COUNT(?review) as ?countReviews
WHERE {
    ?review rdfs:Class schema:UserReview .
    ?review schema:dateCreated ?date .
    BIND (month(?date) as ?month) .
    BIND (year(?date) as ?year) .
    VALUES ?year {2018}
    VALUES ?month {5}
}
GROUP BY ?year ?month
"""

run_query(query=sparql_query, as_dataframe=True)

,year.value,month.value,countReviews.value
0,2018,5,79434


In [ ]:
reviews